# Using oneTBB with SYCL

##### Sections
- [oneTBB Generic Algorithms](#oneTBB-Generic-Algorithms)
- [Calculating pi with tbb::parallel_reduce](#Calculating-pi-with-tbb::parallel_reduce)
- [Using SYCL on GPU and oneTBB on CPU consecutively](#Using-SYCL-on-GPU-and-oneTBB-on-CPU-consecutively)
- [Using tbb::task_group to dispatch GPU and CPU code in parallel](#Using-tbb::task_group-to-dispatch-GPU-and-CPU-code-in-parallel)
- [Using resumable tasks or async_node to share the workload across the CPU and GPU](#Using-resumable-tasks-or-async_node-to-share-the-workload-across-the-CPU-and-GPU)

## Learning Objectives

* Gain experince with oneTBB generic algorithms 
* Use tbb::parallel_reduce to estimate pi as the area of a unit circle
* Learn how to use oneTBB and SYCL together.
* Learn how to use a resumable task or async_node to avoid blocking a oneTBB worker thread.

# oneTBB Generic Algorithms

While it's possible to implement a parallel application by using oneTBB to specify each individual task that can run
concurrently, it is more common to make use of one of its data parallel generic algorithms. The oneTBB library provides 
a number of [generic parallel algorithms](https://spec.oneapi.com/versions/latest/elements/oneTBB/source/algorithms.html),
including `parallel_for`, `parallel_reduce`, `parallel_scan`, `parallel_invoke` and `parallel_pipeline`. These functions 
capture many of the common parallel patterns that are key to unlocking multithreaded performance on the CPU. 

In this section, we provide an exercise that will introduce you one example algorithm, `parallel_reduce`.

## Calculating pi with tbb::parallel_reduce

In this exercise, we calculate pi using the approach shown in the figure below. The idea is to
compute the area of a unit circle, which is equal to pi. We do this by approximating the area of 
1/4th of a unit circle, summing up the areas of ``num_intervals`` rectangles that have
a height of ``sqrt(1-x*x)`` and a width of ``dx == 1.0/num_intervals``. This sum is multiplied by 
4 to compute the total area of the unit circle, providing us with an approximation for pi.

![Algorithm to compute pi](assets/pi.png)

### Run the sequential baseline implementation

Before we add any parallelism, let's validate this approach by running a baseline sequential implementation. Inspect 
the sequential code below - there are no modifications necessary. Run the first cell to create the file, then run the 
cell below it to compile and execute the code. This represents the baseline sequential result and time for our pi 
computation exercise.

1. Inspect the code cell below, then click run ▶ to save the code to a file
2. Run ▶ the cell in the __Build and Run the baseline__ section below the code snippet to compile and execute the code in the saved file

In [ ]:
%%writefile lab/pi-serial.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache-2.0
// =============================================================

#include <chrono>
#include <cmath>
#include <iostream>
#include <limits>

double calc_pi(int num_intervals) {
  double dx = 1.0 / num_intervals;
  double sum = 0.0;
  for (int i = 0; i < num_intervals; ++i) {
    double x = (i+0.5)*dx;
    double h = std::sqrt(1-x*x);
    sum += h*dx;
  }
  double pi = 4 * sum;
  return pi;
}

int main() {
  const int num_intervals = std::numeric_limits<int>::max();
  double serial_time = 0.0;
  {
    auto st0 = std::chrono::high_resolution_clock::now();
    double pi = calc_pi(num_intervals);
    serial_time = 1e-9*(std::chrono::high_resolution_clock::now() - st0).count();
    std::cout << "serial pi == " << pi << std::endl;
  }

  std::cout << "serial_time == " << serial_time << " seconds" << std::endl;
  return 0;
}

### Build and Run the baseline
Select the cell below and click Run ▶ to compile and execute the code above:

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_pi-serial.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_pi-serial.sh; else ./scripts/run_pi-serial.sh; fi

### Implement a parallel version with tbb::parallel_reduce

Our sequential code accumulates values into a single final sum, making it a reduction operation and a match for ``tbb::parallel_reduce``.
You can find detailed documentation for ``parallel_reduce`` [here](https://software.intel.com/content/www/us/en/develop/documentation/tbb-documentation/top/intel-threading-building-blocks-developer-reference/algorithms/parallelreduce-template-function.html). Briefly though, a ``parallel_reduce`` runs a user-provided function
on chunks of the iteration space, potentially concurrently, resulting in several partial results. In our example, these partial results will be partial sums. These partial results are combined using a user-provided reduction function, in our pi example, `std::plus` might be used (hint). 

The interface of ``parallel_reduce`` needed for this example is shown below:

```cpp
template<typename Range, typename Value, typename Func, typename Reduction>
Value parallel_reduce( const Range& range, const Value& identity,
                       const Func& func, const Reduction& reduction );
```

The ``range`` object provides the iteration space, which in our example is 0 to num_intervals - 1. ``identity`` is the identity value for the 
operation that is being parallelized; for a summation, the identity value is 0, since ``sum == sum + 0``. We provide a lambda expression for 
``func`` to compute the partial results, which in our example will return a partial sum for a given range ``r``, accumulating into the 
starting value ``init``. Finally, ``reduction`` is the operation to use to combine the partial results.

For this exercise, complete the following steps:

1. Inspect the code cell below and make the following modifications.
  1. Fix the upper bound in the ``tbb::blocked_range``
  2. Fix the identity value
  3. Add the loop body code
  4. Fix the reduction function
2. When the modifications are complete, click run ▶ to save the code to a file.
3. Run ▶ the cell in the __Build and Run the modified code__ section below the code snippet to compile and execute the code in the saved file.

In [ ]:
%%writefile lab/pi-parallel.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache-2.0
// =============================================================

#include <chrono>
#include <cmath>
#include <iostream>
#include <limits>
#include <thread>
#include <tbb/tbb.h>

#define INCORRECT_VALUE 1
#define INCORRECT_FUNCTION std::minus<double>()

double calc_pi(int num_intervals) {
  double dx = 1.0 / num_intervals;
  double sum = tbb::parallel_reduce(
    /* STEP 1: fix the upper bound: */ tbb::blocked_range<int>(0, INCORRECT_VALUE), 
    /* STEP 2: provide a proper identity value for summation */ INCORRECT_VALUE,
    /* func */ 
    [=](const tbb::blocked_range<int>& r, double init) -> double {
      for (int i = r.begin(); i != r.end(); ++i) {
        // STEP 3: Add the loop body code:
        //         Hint: it will look a lot like the the sequential code.
        //               the returned value should be (init + the_partial_sum)
      }
      return init;
    },
    // STEP 4: provide the reduction function
    //         Hint, maybe std::plus<double>{}
    INCORRECT_FUNCTION
  );
  double pi = 4 * sum;
  return pi;
}

static void warmupTBB() {
  int num_threads = std::thread::hardware_concurrency();
  tbb::parallel_for(0, num_threads,
    [](unsigned int) { 
      std::this_thread::sleep_for(std::chrono::milliseconds(10)); 
  });
}

int main() {
  const int num_intervals = std::numeric_limits<int>::max();
  double parallel_time = 0.0;
  warmupTBB();
  {
    auto pt0 = std::chrono::high_resolution_clock::now();
    double pi = calc_pi(num_intervals);
    parallel_time = 1e-9*(std::chrono::high_resolution_clock::now() - pt0).count();
    std::cout << "parallel pi == " << pi << std::endl;
  }

  std::cout << "parallel_time == " << parallel_time << " seconds" << std::endl;
  return 0;
}

### Build and Run the modified code

Select the cell below and click Run ▶ to compile and execute the code that you modified above:

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_pi-parallel.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_pi-parallel.sh; else ./scripts/run_pi-parallel.sh; fi

### Pi Example Solution (Don't peak, unless you have to)

In [ ]:
%%writefile solutions/pi-parallel.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache-2.0
// =============================================================

#include <chrono>
#include <cmath>
#include <iostream>
#include <limits>
#include <thread>
#include <tbb/tbb.h>

double calc_pi(int num_intervals) {
  double dx = 1.0 / num_intervals;
  double sum = tbb::parallel_reduce(
    /* range = */ tbb::blocked_range<int>(0, num_intervals ), 
    /* identity = */ 0.0,
    /* func */ 
    [=](const tbb::blocked_range<int>& r, double init) -> double {
      for (int i = r.begin(); i != r.end(); ++i) {
        double x = (i+0.5)*dx;
        double h = std::sqrt(1-x*x);
        init += h*dx;
      }
      return init;
    },
    std::plus<double>{}
  );
  double pi = 4 * sum;
  return pi;
}

static void warmupTBB() {
  int num_threads = std::thread::hardware_concurrency();
  tbb::parallel_for(0, num_threads,
    [](unsigned int) { 
      std::this_thread::sleep_for(std::chrono::milliseconds(10)); 
  });
}

int main() {
  const int num_intervals = std::numeric_limits<int>::max();
  double parallel_time = 0.0;
  warmupTBB();
  {
    auto pt0 = std::chrono::high_resolution_clock::now();
    double pi = calc_pi(num_intervals);
    parallel_time = 1e-9*(std::chrono::high_resolution_clock::now() - pt0).count();
    std::cout << "parallel pi == " << pi << std::endl;
  }

  std::cout << "parallel_time == " << parallel_time << " seconds" << std::endl;
  return 0;
}


In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_pi-solution.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_pi-solution.sh; else ./scripts/run_pi-solution.sh; fi

# Using SYCL on GPU and oneTBB on CPU consecutively

Now we can look at using oneTBB algorithms in combination with SYCL. 

Let's start by computing `c = a + alpha * b` (usually known as a `triad` operation), first using a SYCL parallel_for and then a TBB parallel_for. On the **GPU**, we compute `c_sycl = a_array + b_array * alpha`, whereas on the **CPU**, we write to a different result array and compute `c_tbb = a_array + b_array * alpha`. In this example, we are executing these algorithms one after the other, and not overlapping the use of the GPU with the use of the CPU.

<img src="assets/Triad-GPU-then-CPU.png" width="1000">


1. Inspect the code cell below, then click run ▶ to save the code to a file
2. Run ▶ the cell in the __Build and Run the baseline__ section below the code snippet to compile and execute the code in the saved file

In [ ]:
%%writefile lab/triad-consecutive.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache 2.0
// =============================================================

#include "../common/common_utils.hpp"
#include <array>
#include <tbb/blocked_range.h>
#include <tbb/parallel_for.h>

int main() {
  const float alpha = 0.5;  // alpha for triad calculation
  const size_t array_size = 16;

  std::array<float, array_size> a_array, b_array, c_sycl, c_tbb;

  // sets array values to 0..N
  common::init_input_arrays(a_array, b_array); 

  std::cout << "executing on the GPU using SYCL\n";
  {  
    sycl::buffer a_buffer{a_array}, b_buffer{b_array}, c_buffer{c_sycl};
    sycl::queue q{sycl::default_selector{}};
    q.submit([&](sycl::handler& h) {            
      sycl::accessor a_accessor{a_buffer, h, sycl::read_only};
      sycl::accessor b_accessor{b_buffer, h, sycl::read_only};
      sycl::accessor c_accessor{c_buffer, h, sycl::write_only};
      h.parallel_for(sycl::range<1>{array_size}, [=](sycl::id<1> index) {
         c_accessor[index] = a_accessor[index] + b_accessor[index] * alpha;
      });  
    }).wait(); //Wait here
  }

  std::cout << "executing on the CPU using TBB\n";
  tbb::parallel_for(tbb::blocked_range<int>(0, a_array.size()),
    [&](tbb::blocked_range<int> r) {
      for (int index = r.begin(); index < r.end(); ++index) {
        c_tbb[index] = a_array[index] + b_array[index] * alpha;
      }
  });

  common::validate_results(alpha, a_array, b_array, c_sycl, c_tbb);
  common::print_results(alpha, a_array, b_array, c_sycl, c_tbb);
} 

### Build and Run the baseline
Select the cell below and click Run ▶ to compile and execute the code above:

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_consecutive.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_consecutive.sh; else ./scripts/run_consecutive.sh; fi

# Using tbb::task_group to dispatch GPU and CPU code in parallel

Of course the CPU and the GPU can work in parallel. Our first approach will be to use `tbb::task_group` to spawn a task for the GPU and another concurrent one for the CPU. It will look like:

<img src="assets/Triad-task_group.png" width="1000">

The class `tbb::task_group` is quite easy to use:

```
tbb::task_group g; // Create a task_group object g
g.run([]{cout << "One task passed to g.run as a lambda\n";});
g.run([]{cout << "Another concurrent task in this lambda\n";});
g.wait() // Wait for both tasks to complete
```

For this exercise, complete the following steps:

1. Inspect the code cell below and make the following modifications.
  1. Complete the body of the lambda for the first call to run, offloading the code to the GPU
  2. Complete the body of the lambda for the second call to run, executing the code on the CPU 
2. When the modifications are complete, click run ▶ to save the code to a file.
3. Run ▶ the cell in the __Build and Run the modified code__ section below the code snippet to compile and execute the code in the saved file.

In [ ]:
%%writefile lab/triad-task_group.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache 2.0
// =============================================================

#include "../common/common_utils.hpp"
#include <array>
#include <tbb/blocked_range.h>
#include <tbb/parallel_for.h>
#include "tbb/task_group.h"

int main() {
  const float alpha = 0.5;  // coeff for triad calculation
  const size_t array_size = 16;

  std::array<float, array_size> a_array, b_array, c_sycl, c_tbb;

  // sets array values to 0..N
  common::init_input_arrays(a_array, b_array); 

  // create task_group
  tbb::task_group tg;

  // Run a TBB task that uses SYCL to offload to GPU, function run does not block
  tg.run([&, alpha]() {
    std::cout << "executing on the GPU using SYCL\n";
    {  
      // STEP A: Complete the body to offload to the GPU
      //         Hint: look at (copy from) the consecutive calls sample
    }
  });

  // Run a TBB task that uses SYCL to offload to CPU
  tg.run([&, alpha]() {
    std::cout << "executing on the CPU using TBB\n";
    // STEP B: Complete the body to offload to the CPU
    //         Hint: look at (copy from) the consecutive calls sample
  });

  // wait for both TBB tasks to complete
  tg.wait();

  common::validate_results(alpha, a_array, b_array, c_sycl, c_tbb);
  common::print_results(alpha, a_array, b_array, c_sycl, c_tbb);
} 

### Build and Run the modified code

Select the cell below and click Run ▶ to compile and execute the code that you modified above:

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_tasks.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_tasks.sh; else ./scripts/run_tasks.sh; fi

### Solution (Don't peak unless you have to)

In [ ]:
%%writefile solutions/triad-task_group-solved.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache 2.0
// =============================================================

#include "../common/common_utils.hpp"
#include <array>
#include <tbb/blocked_range.h>
#include <tbb/parallel_for.h>
#include "tbb/task_group.h"

int main() {
  const float alpha = 0.5;  // coeff for triad calculation
  const size_t array_size = 16;

  std::array<float, array_size> a_array, b_array, c_sycl, c_tbb;

  // sets array values to 0..N
  common::init_input_arrays(a_array, b_array); 

  // create task_group
  tbb::task_group tg;

  // Run a TBB task that uses SYCL to offload to GPU, function run does not block
  tg.run([&, alpha]() {
    std::cout << "executing on the GPU using SYCL\n";
    {  
      sycl::buffer a_buffer{a_array}, b_buffer{b_array}, c_buffer{c_sycl};
      sycl::queue q{sycl::default_selector{}};
      q.submit([&](sycl::handler& h) {            
        sycl::accessor a_accessor{a_buffer, h, sycl::read_only};
        sycl::accessor b_accessor{b_buffer, h, sycl::read_only};
        sycl::accessor c_accessor{c_buffer, h, sycl::write_only};
        h.parallel_for(sycl::range<1>{array_size}, [=](sycl::id<1> index) {
           c_accessor[index] = a_accessor[index] + b_accessor[index] * alpha;
        });  
      }).wait();
    }
  });

  // Run a TBB task that uses SYCL to offload to CPU
  tg.run([&, alpha]() {
    std::cout << "executing on the CPU using TBB\n";

    tbb::parallel_for(tbb::blocked_range<int>(0, a_array.size()),
      [&](tbb::blocked_range<int> r) {
        for (int index = r.begin(); index < r.end(); ++index) {
          c_tbb[index] = a_array[index] + b_array[index] * alpha;
        }
    });
  });

  // wait for both TBB tasks to complete
  tg.wait();

  common::validate_results(alpha, a_array, b_array, c_sycl, c_tbb);
  common::print_results(alpha, a_array, b_array, c_sycl, c_tbb);
} 

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_tasks-solved.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_tasks-solved.sh; else ./scripts/run_tasks-solved.sh; fi

### 

# Using resumable tasks or async_node to share the workload across the CPU and GPU

Let's say we only have to compute a single result array. But, we want to get the most out of both the CPU and the GPU by sharing the workload. The most simple alternative is to statically partition the iteration space in two sub-regions and assign the first partition to the GPU and the second one to the CPU:

<img src="assets/Triad-Suspend_task.png" width="500">

In the next code we introduce several changes:

1. We use the `offload_ratio=0.5` variable to indicate that we want to offload to the GPU (using a SYCL queue) 50% of the iteration space and the other 50% to the CPU (that gets processed by a `tbb::parallel_for`)
2. We use a different `alpha` for the GPU (`alpha_sycl = 0.5`) and for the CPU (`alpha_tbb = 1.0`). That way, when printing the C array we can easily identify the sub-array updated on the GPU (the *.5 values) and the sub-array updated on the CPU (all integer values).
3. We use USM host-allocated arrays (also accessible from the GPU), instead of using sycl::buffer. That way: i) we provide another example that uses USM; ii) the resulting code is simpler; and iii) it may exhibit performance improvements on integrated GPUs that share the global memory with the CPU.
4. We use two C arrays (`c_sycl` and `c_tbb`) as in the previous examples, but after the GPU and the CPU are done with their respective duties, we combine the GPU part into the CPU array `c_tbb`). In some cases (USM with fine-grained sharing capabilities) a single C array would do, but for portability sake, we decided to use the safest approach that avoids having the CPU and the GPU concurrently writing in the same array (even if it is in different non-overlapping regions).

This is a simple fine-grained CPU+GPU demonstration that may not perform better than a CPU-only or GPU-only alternative, but for other coarser-grained problems this static partitioning of the iteration space can improve performance and/or reduce energy consumption.

## Resumable tasks

In the next code, we use `tbb::task::suspend()` instead of `tbb::task_group::run()` to avoid blocking a TBB working thread while waiting for the GPU task. Here you can find detailed information about [tbb::suspend_task](https://www.threadingbuildingblocks.org/docs/help/reference/appendices/preview_features/resumable_tasks.html), but you can also refer to slide 27 of the previous presentation.

In the current state, a user-defined `AsyncActivity`is created in the `main()` function. At construction time, AsyncActity starts a thread that waits until `submit_flag==true`, then offloads the computation to the GPU, and when the GPU has finished, it sets `submit_flag=false`. `AsyncActivity::submit()` is called in the `main()` function after starting the CPU computation. This member function is the one setting `submit_flat=true` and then spin-waits until the thread completes the GPU work and sets `submit_flag=false`. There is useless thread spinning here, so let's fix it and simplify it using `tbb::task::suspend()`.

1. Inspect the code cell below and make the following modifications.
  1. STEP A: Inside `main()`, put the call to submit inside of a call to `tbb::task::suspend()`, as in Slide 27 from the previous presentation
  2. STEP B: Inside the thread body, remove the `submit_flag=false` and instead use `tbb::task::resume()`.
  3. STEP C: Inside `AsyncActivity::submit()` remove the idle spin loop waiting for the GPU to finish (now `tbb::task::suspend()` takes care of waiting)
2. Run ▶ the cell in the __Build and Run the modified code__ section below the code snippet to compile and execute the code in the saved file

In [ ]:
%%writefile lab/triad-hetero-suspend-resume.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache 2.0
// =============================================================

#include "../common/common_utils.hpp"

#include <array>
#include <atomic>
#include <cmath>
#include <iostream>
#include <thread>
#include <algorithm>

#include <CL/sycl.hpp>

#include <tbb/blocked_range.h>
#include <tbb/task.h>
#include <tbb/task_group.h>
#include <tbb/parallel_for.h>

template<size_t array_size>
class AsyncActivity {
    float alpha;
    const float *a_array, *b_array; 
    float *c_sycl;
    sycl::queue& q;
    float offload_ratio;
    std::atomic<bool> submit_flag;
    tbb::task::suspend_point suspend_point;
    std::thread service_thread;

public:
    AsyncActivity(float alpha_sycl, const float *a, const float *b, float *c, sycl::queue &queue) : 
      alpha{alpha_sycl}, a_array{a}, b_array{b}, c_sycl{c}, q{queue}, 
      offload_ratio{0}, submit_flag{false},
      service_thread([this] {
        // We are in the constructor so this thread is dispatched at AsyncActivity construction
        // Wait until the job is submitted into the tbb::suspend_task()
        while(!submit_flag) std::this_thread::yield();
        // Here submit_flag==true --> DISPATCH GPU computation
        std::size_t array_size_sycl = std::ceil(array_size * offload_ratio);
        float l_alpha=alpha;
        const float *la=a_array, *lb=b_array;
        float *lc=c_sycl;
        q.submit([&](sycl::handler& h) {            
            h.parallel_for(sycl::range<1>{array_size_sycl}, [=](sycl::id<1> index) {
              lc[index] = la[index] + lb[index] * l_alpha;
            });  
        }).wait(); //The thread may spin or block here.
  
        // Pass a signal into the main thread that the GPU work is completed
        // STEP B: remove the submit_flag=false and instead use tbb::task::resume(). 
        // See https://www.threadingbuildingblocks.org/docs/help/reference/appendices/preview_features/resumable_tasks.html
        submit_flag = false;
      }) {}

    ~AsyncActivity() {
        service_thread.join();
    }

    void submit( float ratio, tbb::task::suspend_point sus_point ) {
        offload_ratio = ratio;
        suspend_point = sus_point;
        submit_flag = true;
        // STEP C: remove the idle spin loop on the submit_flag
        //         this becomes unecessary once suspend / resume is used
        // Now it is necessary to avoid this function returning befor the GPU has finished
        while (submit_flag) // Wait until submit_flat==false (The service thread does that after the GPU has finished)
          std::this_thread::yield();        
    }
}; // class AsyncActivity

int main() {
  
  constexpr float ratio = 0.5; // CPU or GPU offload ratio
  // We use different alpha coefficients so that 
  //we can identify the GPU and CPU part if we print c_array result
  const float alpha_sycl = 0.5, alpha_tbb = 1.0;  
  constexpr size_t array_size = 16;

  sycl::queue q{sycl::gpu_selector{}};
  std::cout << "Using device: " << q.get_device().get_info<sycl::info::device::name>() << '\n'; 

  //This host allocation of c comes handy specially for integrated GPUs (CPU and GPU share mem)
  float *a_array = malloc_host<float>(array_size, q); 
  float *b_array = malloc_host<float>(array_size, q); 
  float *c_sycl = malloc_host<float>(array_size, q);
  float *c_tbb = new float[array_size];

  // sets array values to 0..N
  std::iota(a_array, a_array+array_size,0); 
  std::iota(b_array, b_array+array_size,0);
  
  tbb::task_group tg;
  AsyncActivity<array_size> activity{alpha_sycl, a_array, b_array, c_sycl, q};

  //Spawn a task that runs a parallel_for on the CPU
  tg.run([&, alpha_tbb]{
   std::size_t i_start = static_cast<std::size_t>(std::ceil(array_size * ratio));
   std::size_t i_end = array_size;
   tbb::parallel_for(i_start, i_end, [=]( std::size_t index ) {
     c_tbb[index] = a_array[index] + alpha_tbb * b_array[index];
   });
  });

  //Spawn another task that asyncrhonously offloads computation to the GPU  
  // STEP A: Put the call to submit inside of a call to tbb::task::suspend, as in Slide 27 from the previous presentation
  activity.submit(ratio, tbb::task::suspend_point{});

  tg.wait();

  //Merge GPU result into CPU array
  std::size_t gpu_end = static_cast<std::size_t>(std::ceil(array_size * ratio));
  std::copy(c_sycl, c_sycl+gpu_end, c_tbb);

  common::validate_usm_results(ratio, alpha_sycl, alpha_tbb, a_array, b_array, c_tbb, array_size);
  if(array_size<64)
    common::print_usm_results(ratio, alpha_sycl, alpha_tbb, a_array, b_array, c_tbb, array_size);

  free(a_array,q);
  free(b_array,q);
  free(c_sycl,q);
  delete[] c_tbb;
}

### Build and Run the modified code
Select the cell below and click Run ▶ to compile and execute the code above:

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_suspend-resume.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_suspend-resume.sh; else ./scripts/run_suspend-resume.sh; fi

### Solution (Don't peak unless you have to)

In [ ]:
%%writefile solutions/triad-hetero-suspend-resume-solved.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache 2.0
// =============================================================

#include "../common/common_utils.hpp"

#include <array>
#include <atomic>
#include <cmath>
#include <iostream>
#include <thread>
#include <algorithm>

#include <CL/sycl.hpp>

#include <tbb/blocked_range.h>
#include <tbb/task.h>
#include <tbb/task_group.h>
#include <tbb/parallel_for.h>

template<size_t array_size>
class AsyncActivity {
    float alpha;
    const float *a_array, *b_array; 
    float *c_sycl;
    sycl::queue& q;
    float offload_ratio;
    std::atomic<bool> submit_flag;
    tbb::task::suspend_point suspend_point;
    std::thread service_thread;

public:
    AsyncActivity(float alpha_sycl, const float *a, const float *b, float *c, sycl::queue &queue) : 
      alpha{alpha_sycl}, a_array{a}, b_array{b}, c_sycl{c}, q{queue}, 
      offload_ratio{0}, submit_flag{false},
      service_thread([this] {
        // Wait until the job will be submitted into the async activity
        while(!submit_flag) std::this_thread::yield();
        // Here submit_flag==true --> DISPATCH GPU computation
        std::size_t array_size_sycl = std::ceil(array_size * offload_ratio);
        float l_alpha=alpha;
        const float *la=a_array, *lb=b_array;
        float *lc=c_sycl;
        q.submit([&](sycl::handler& h) {            
            h.parallel_for(sycl::range<1>{array_size_sycl}, [=](sycl::id<1> index) {
              lc[index] = la[index] + lb[index] * l_alpha;
            });  
        }).wait(); //The thread may spin or block here.
  
        // Pass a signal into the main thread that the GPU work is completed
        tbb::task::resume(suspend_point);
      }) {}

    ~AsyncActivity() {
        service_thread.join();
    }

    void submit( float ratio, tbb::task::suspend_point sus_point ) {
        offload_ratio = ratio;
        suspend_point = sus_point;
        submit_flag = true;
    }
}; // class AsyncActivity

int main() {
  
  constexpr float ratio = 0.5; // CPU or GPU offload ratio
  // We use different alpha coefficients so that 
  //we can identify the GPU and CPU part if we print c_array result
  const float alpha_sycl = 0.5, alpha_tbb = 1.0;  
  constexpr size_t array_size = 16;

  sycl::queue q{sycl::gpu_selector{}};
  std::cout << "Using device: " << q.get_device().get_info<sycl::info::device::name>() << '\n'; 

  //This host allocation of c comes handy specially for integrated GPUs (CPU and GPU share mem)
  float *a_array = malloc_host<float>(array_size, q); 
  float *b_array = malloc_host<float>(array_size, q); 
  float *c_sycl = malloc_host<float>(array_size, q);
  float *c_tbb = new float[array_size];

  // sets array values to 0..N
  std::iota(a_array, a_array+array_size,0); 
  std::iota(b_array, b_array+array_size,0);
  
  tbb::task_group tg;
  AsyncActivity<array_size> activity{alpha_sycl, a_array, b_array, c_sycl, q};

  //Spawn a task that runs a parallel_for on the CPU
  tg.run([&, alpha_tbb]{
   std::size_t i_start = static_cast<std::size_t>(std::ceil(array_size * ratio));
   std::size_t i_end = array_size;
   tbb::parallel_for(i_start, i_end, [=]( std::size_t index ) {
     c_tbb[index] = a_array[index] + alpha_tbb * b_array[index];
   });
  });

  //Spawn another task that asyncrhonously offloads computation to the GPU  
    tbb::task::suspend([&]( tbb::task::suspend_point suspend_point ) {
     activity.submit(ratio, suspend_point);
    });

  tg.wait();

  //Merge GPU result into CPU array
  std::size_t gpu_end = static_cast<std::size_t>(std::ceil(array_size * ratio));
  std::copy(c_sycl, c_sycl+gpu_end, c_tbb);

  common::validate_usm_results(ratio, alpha_sycl, alpha_tbb, a_array, b_array, c_tbb, array_size);
  if(array_size<64)
    common::print_usm_results(ratio, alpha_sycl, alpha_tbb, a_array, b_array, c_tbb, array_size);

  free(a_array,q);
  free(b_array,q);
  free(c_sycl,q);
  delete[] c_tbb;
}

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_suspend-resume-solved.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_suspend-resume-solved.sh; else ./scripts/run_suspend-resume-solved.sh; fi

## Using flow::async_node

Now let's assume we have a stream of data that is going to be processed in a TBB Flow Graph, and so use a `tbb::task::async_node` instead. As you can see in the figure, our graph has several nodes:

<img src="assets/Triad-Async_task.png" width="500">

1. The `tbb::flow::input_node` (**in_node**) initializes a struct with the arrays and companion information, initializes A and B, and passes a pointer to that structure to two nodes that will process the arrays in parallel.
2. The `tbb::flow::function_node` (**cpu_node**) computes a sub-region of the arrays on the CPU, using a nested `tbb::parallel_for` to distribute the CPU load among the available CPU cores.
3. The `tbb::flow::async_node` (**a_node**) dispatches to an AsyncActivity, quite similar to the previous example. As a reference, you can look at the [reference of tbb::flow::async_node](https://www.threadingbuildingblocks.org/docs/help/index.htm#reference/appendices/preview_features/resumable_tasks.html) or at an easier [example](https://link.springer.com/chapter/10.1007/978-1-4842-4398-5_18).
4. The `tbb::flow::join_node` (**node_join**) waits until the CPU and the GPU are done.
5. The `tbb::flow::function_node` (**out_node**) receives the pointer to the message that contains the resulting array that is checked and printed.

In the following code, the `AsyncActivity` wastes a TBB working thread by spinnning until the GPU has finished processing its region of the arrays, much like in the previous exercise. We can certainly do it better:

1. Inspect the code cell below and make the following modifications.
  1. STEP A: Inside `main()`, in the body of the `a_node`, remove the `try_put` that in this code is necessary to keep it working (it sends a message to the `node_join`). This `try_put` should now be moved to `AsyncActivity` thread, as we do in the next STEP.
  2. STEP B: Inside the `AsyncActivity` thread body, remove the `submit_flag=false` and instead use `gateway->try_put(msg)`. We also have to call `gateway->release_wait()` so that we inform the graph, `g`, that there is no need to wait any longer for the `AsyncActivity`.
  3. STEP C: Inside `AsyncActivity::submit()` add a call to `gateway.reserve_wait()` to notify the graph that you are dispatching to an asynchronous activity and that the graph has to wait for it.
  4. STEP D: Inside `AsyncActivity::submit()` remove the idle spin loop waiting for the GPU to finish (now the `reserve_wait/release_wait` pair takes care of the necessary synchronization).
2. Run ▶ the cell in the __Build and Run the modified code__ section below the code snippet to compile and execute the code in the saved file

In [ ]:
%%writefile lab/triad-hetero-async_node.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache 2.0
// =============================================================

#include "../common/common_utils.hpp"

#include <cmath>  //for std::ceil
#include <array>
#include <atomic>
#include <iostream>
#include <thread>

#include <CL/sycl.hpp>

#include <tbb/blocked_range.h>
#include <tbb/flow_graph.h>
#include <tbb/global_control.h>
#include <tbb/parallel_for.h>

constexpr size_t array_size = 16;

template<size_t ARRAY_SIZE>
struct msg_t {
  static constexpr size_t array_size = ARRAY_SIZE;
  const float offload_ratio = 0.5;
  const float alpha_0 = 0.5;
  const float alpha_1 = 1.0;
  std::array<float, array_size> a_array;  // input
  std::array<float, array_size> b_array;  // input
  std::array<float, array_size> c_sycl;   // GPU output
  std::array<float, array_size> c_tbb;    // CPU output
};

using msg_ptr = std::shared_ptr<msg_t<array_size>>;

using async_node_t = tbb::flow::async_node<msg_ptr, msg_ptr>;
using gateway_t = async_node_t::gateway_type;

class AsyncActivity {
  msg_ptr msg;
  gateway_t* gateway_ptr;
  std::atomic<bool> submit_flag;
  std::thread service_thread;

 public:
  AsyncActivity() : msg{nullptr}, gateway_ptr{nullptr}, submit_flag{false},
    service_thread( [this] {
      //Wait until other thread sets submit_flag=true
      while( !submit_flag ) std::this_thread::yield();
      // Here we go! Dispatch code to the GPU
      // Execute the kernel over a portion of the array range
      size_t array_size_sycl = std::ceil(msg->a_array.size() * msg->offload_ratio);
      {  
        sycl::buffer a_buffer{msg->a_array}, b_buffer{msg->b_array}, c_buffer{msg->c_sycl};
        sycl::queue q{sycl::gpu_selector{}};
        float alpha = msg->alpha_0;
        q.submit([&, alpha](sycl::handler& h) {            
          sycl::accessor a_accessor{a_buffer, h, sycl::read_only};
          sycl::accessor b_accessor{b_buffer, h, sycl::read_only};
          sycl::accessor c_accessor{c_buffer, h, sycl::write_only};
          h.parallel_for(sycl::range<1>{array_size_sycl}, [=](sycl::id<1> index) {
            c_accessor[index] = a_accessor[index] + b_accessor[index] * alpha;
          });  
        }).wait();
      }
      // STEP B: Remove the set of submit_flag and replace with
      //         a call to try_put on the gateway
      //         and a call to release_wait on the gateway
      submit_flag = false;
    } ) {}

  ~AsyncActivity() {
    service_thread.join();
  }

  void submit(msg_ptr m, gateway_t& gateway) {
    // STEP C: add a call to gateway.reserve_wait()
    msg = m;
    gateway_ptr = &gateway;
    submit_flag = true;
    // STEP D: remove the idle spin loop on the submit_flag
    //         this becomes unecessary once reserve_wait / release_wait is used
    while (submit_flag)
      std::this_thread::yield();
  }
};

int main() {
  tbb::flow::graph g;

  // Input node:
  tbb::flow::input_node<msg_ptr> in_node{g, 
    [&](tbb::flow_control& fc) -> msg_ptr {
      static bool has_run = false;
      if (has_run) fc.stop();
      has_run = true; // This example only creates a message to feed the Flow Graph
      msg_ptr msg = std::make_shared<msg_t<array_size>>();
      common::init_input_arrays(msg->a_array, msg->b_array);
      return msg;
    }
  };

  // CPU node
  tbb::flow::function_node<msg_ptr, msg_ptr> cpu_node{
      g, tbb::flow::unlimited, [&](msg_ptr msg) -> msg_ptr {
        size_t i_start = static_cast<size_t>(std::ceil(msg->array_size * msg->offload_ratio));
        size_t i_end = static_cast<size_t>(msg->array_size);
        auto &a_array = msg->a_array, &b_array = msg->b_array, &c_tbb = msg->c_tbb;
        float alpha = msg->alpha_1;
        tbb::parallel_for(tbb::blocked_range<size_t>{i_start, i_end},
          [&, alpha](const tbb::blocked_range<size_t>& r) {
            for (size_t i = r.begin(); i < r.end(); ++i)
              c_tbb[i] = a_array[i] + alpha * b_array[i];
            }
        );
        return msg;
      }};

  // async node -- GPU
  AsyncActivity async_act;
  async_node_t a_node{g, tbb::flow::unlimited,
    [&async_act](msg_ptr msg, gateway_t& gateway) {
      async_act.submit(msg, gateway);
      // STEP A: remove the try_put below since submit will not block
      //         In STEP B you will modify AsyncActivity so that it makes the call to try_put instead
      gateway.try_put(msg);
    }
  };

  // join node
  using join_t = tbb::flow::join_node<std::tuple<msg_ptr, msg_ptr>>;
  join_t node_join{g};

  // out node
  tbb::flow::function_node<join_t::output_type> out_node{g, tbb::flow::unlimited, 
    [&](const join_t::output_type& two_msgs) {
      msg_ptr msg = std::get<0>(two_msgs); //Both msg's point to the same data
      //Merge GPU result into CPU array
      std::size_t gpu_end = static_cast<std::size_t>(std::ceil(msg->array_size * msg->offload_ratio));
      std::copy(msg->c_sycl.begin(), msg->c_sycl.begin()+gpu_end, msg->c_tbb.begin());
      common::validate_hetero_results(msg->offload_ratio, msg->alpha_0, msg->alpha_1, 
                                      msg->a_array, msg->b_array, msg->c_tbb);
      if(msg->array_size<=64)
        common::print_hetero_results(msg->offload_ratio, msg->alpha_0, msg->alpha_1, 
                                     msg->a_array, msg->b_array, msg->c_tbb);
    }
  };  // end of out node

  // construct graph
  tbb::flow::make_edge(in_node, a_node);
  tbb::flow::make_edge(in_node, cpu_node);
  tbb::flow::make_edge(a_node, tbb::flow::input_port<0>(node_join));
  tbb::flow::make_edge(cpu_node, tbb::flow::input_port<1>(node_join));
  tbb::flow::make_edge(node_join, out_node);

  in_node.activate();
  g.wait_for_all();

  return 0;
}

### Build and Run the modified code

Select the cell below and click Run ▶ to compile and execute the code that you modified above:

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_async_node.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_async_node.sh; else ./scripts/run_async_node.sh; fi

### Solution (Don't peak unless you have to)

In [ ]:
%%writefile solutions/triad-hetero-async_node-solved.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache 2.0
// =============================================================

#include "../common/common_utils.hpp"

#include <cmath>  //for std::ceil
#include <array>
#include <atomic>
#include <iostream>
#include <thread>

#include <CL/sycl.hpp>

#include <tbb/blocked_range.h>
#include <tbb/flow_graph.h>
#include <tbb/global_control.h>
#include <tbb/parallel_for.h>

constexpr size_t array_size = 16;

template<size_t ARRAY_SIZE>
struct msg_t {
  static constexpr size_t array_size = ARRAY_SIZE;
  const float offload_ratio = 0.5;
  const float alpha_0 = 0.5;
  const float alpha_1 = 1.0;
  std::array<float, array_size> a_array;  // input
  std::array<float, array_size> b_array;  // input
  std::array<float, array_size> c_sycl;   // GPU output
  std::array<float, array_size> c_tbb;    // CPU output
};

using msg_ptr = std::shared_ptr<msg_t<array_size>>;

using async_node_t = tbb::flow::async_node<msg_ptr, msg_ptr>;
using gateway_t = async_node_t::gateway_type;

class AsyncActivity {
  msg_ptr msg;
  gateway_t* gateway_ptr;
  std::atomic<bool> submit_flag;
  std::thread service_thread;

 public:
  AsyncActivity() : msg{nullptr}, gateway_ptr{nullptr}, submit_flag{false},
    service_thread( [this] {
      //Wait until other thread sets submit_flag=true
      while( !submit_flag ) std::this_thread::yield();
      // Here we go! Dispatch code to the GPU
      // Execute the kernel over a portion of the array range
      size_t array_size_sycl = std::ceil(msg->a_array.size() * msg->offload_ratio);
      {  
        sycl::buffer a_buffer{msg->a_array}, b_buffer{msg->b_array}, c_buffer{msg->c_sycl};
        sycl::queue q{sycl::gpu_selector{}};
        float alpha = msg->alpha_0;
        q.submit([&, alpha](sycl::handler& h) {            
          sycl::accessor a_accessor{a_buffer, h, sycl::read_only};
          sycl::accessor b_accessor{b_buffer, h, sycl::read_only};
          sycl::accessor c_accessor{c_buffer, h, sycl::write_only};
          h.parallel_for(sycl::range<1>{array_size_sycl}, [=](sycl::id<1> index) {
            c_accessor[index] = a_accessor[index] + b_accessor[index] * alpha;
          });  
        }).wait();
      }
      gateway_ptr->try_put(msg);
      gateway_ptr->release_wait();
    } ) {}

  ~AsyncActivity() {
    service_thread.join();
  }

  void submit(msg_ptr m, gateway_t& gateway) {
    gateway.reserve_wait();
    msg = m;
    gateway_ptr = &gateway;
    submit_flag = true;
  }
};

int main() {
  tbb::flow::graph g;

  // Input node:
  tbb::flow::input_node<msg_ptr> in_node{g, 
    [&](tbb::flow_control& fc) -> msg_ptr {
      static bool has_run = false;
      if (has_run) fc.stop();
      has_run = true; // This example only creates a message to feed the Flow Graph
      msg_ptr msg = std::make_shared<msg_t<array_size>>();
      common::init_input_arrays(msg->a_array, msg->b_array);
      return msg;
    }
  };

  // CPU node
  tbb::flow::function_node<msg_ptr, msg_ptr> cpu_node{
      g, tbb::flow::unlimited, [&](msg_ptr msg) -> msg_ptr {
        size_t i_start = static_cast<size_t>(std::ceil(msg->array_size * msg->offload_ratio));
        size_t i_end = static_cast<size_t>(msg->array_size);
        auto &a_array = msg->a_array, &b_array = msg->b_array, &c_tbb = msg->c_tbb;
        float alpha = msg->alpha_1;
        tbb::parallel_for(tbb::blocked_range<size_t>{i_start, i_end},
          [&, alpha](const tbb::blocked_range<size_t>& r) {
            for (size_t i = r.begin(); i < r.end(); ++i)
              c_tbb[i] = a_array[i] + alpha * b_array[i];
            }
        );
        return msg;
      }};

  // async node -- GPU
  AsyncActivity async_act;
  async_node_t a_node{g, tbb::flow::unlimited,
    [&async_act](msg_ptr msg, gateway_t& gateway) {
      async_act.submit(msg, gateway);
    }
  };

  // join node
  using join_t = tbb::flow::join_node<std::tuple<msg_ptr, msg_ptr>>;
  join_t node_join{g};

  // out node
  tbb::flow::function_node<join_t::output_type> out_node{g, tbb::flow::unlimited, 
    [&](const join_t::output_type& two_msgs) {
      msg_ptr msg = std::get<0>(two_msgs); //Both msg's point to the same data
      //Merge GPU result into CPU array
      std::size_t gpu_end = static_cast<std::size_t>(std::ceil(msg->array_size * msg->offload_ratio));
      std::copy(msg->c_sycl.begin(), msg->c_sycl.begin()+gpu_end, msg->c_tbb.begin());
      common::validate_hetero_results(msg->offload_ratio, msg->alpha_0, msg->alpha_1, 
                                      msg->a_array, msg->b_array, msg->c_tbb);
      if(msg->array_size<=64)
        common::print_hetero_results(msg->offload_ratio, msg->alpha_0, msg->alpha_1, 
                                     msg->a_array, msg->b_array, msg->c_tbb);
    }
  };  // end of out node

  // construct graph
  tbb::flow::make_edge(in_node, a_node);
  tbb::flow::make_edge(in_node, cpu_node);
  tbb::flow::make_edge(a_node, tbb::flow::input_port<0>(node_join));
  tbb::flow::make_edge(cpu_node, tbb::flow::input_port<1>(node_join));
  tbb::flow::make_edge(node_join, out_node);

  in_node.activate();
  g.wait_for_all();

  return 0;
}

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_async_node-solved.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_async_node-solved.sh; else ./scripts/run_async_node-solved.sh; fi

### Congrats on getting here!